In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

data = pd.read_csv('processed_data.csv')

In [3]:
patient_counts = data['patient_id'].value_counts()
patient_counts_pd = pd.DataFrame(patient_counts)
patient_counts_pd = patient_counts_pd.loc[patient_counts_pd['count'] > 29]

filtered_data = data[data['patient_id'].isin(patient_counts_pd.index)]  # 해당 ID만 남기기
filtered_data.to_csv('filtered_id_data.csv')

In [6]:
threshold_high = 25.34 

threshold_low = -2.61

In [12]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

for pid, group in filtered_data.groupby('patient_id'):

    # feature / target 나누기
    y = group['sleep_quality_score']
    X = group.drop(columns=['sleep_quality_score', 'score_level', 'patient_id', 'timestamp'])
    

    # train/test split (시계열 고려해서 shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    # Lasso 모델 학습
    model = Lasso(alpha=0.1, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test) 

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'[id={pid}] MSE: {mse:.3f}, R²: {r2:.3f} x_train_count: {len(X_train)}')

    y_test_pd = pd.DataFrame(y_test,columns=['sleep_quality_score'])
    y_pred_pd = pd.DataFrame(y_pred,columns=['sleep_quality_score'])

    
    # 등급 부여
    y_pred_pd['score_level'] = 'mid'
    y_pred_pd.loc[y_pred_pd['sleep_quality_score'] >= threshold_high, 'score_level'] = 'good'
    y_pred_pd.loc[y_pred_pd['sleep_quality_score'] <= threshold_low, 'score_level'] = 'bad'
    
    y_test_pd['score_level'] = 'mid'
    y_test_pd.loc[y_test_pd['sleep_quality_score'] >= threshold_high, 'score_level'] = 'good'
    y_test_pd.loc[y_test_pd['sleep_quality_score'] <= threshold_low, 'score_level'] = 'bad'
    
    # confusion matrix 계산
    labels = ['bad', 'mid', 'good']  # 클래스 순서 정의
    cm = confusion_matrix(y_test_pd['score_level'], y_pred_pd['score_level'], labels=labels)
    
    # DataFrame으로 보기 좋게 출력
    cm_df = pd.DataFrame(cm, index=[f"True_{l}" for l in labels], columns=[f"Pred_{l}" for l in labels])
    print("Confusion Matrix:")
    print(cm_df)

        # 성능 리포트 (정밀도, 재현율 등)
    print("\nClassification Report:")
    print(classification_report(y_test_pd['score_level'], y_pred_pd['score_level'], labels=labels))


[id=0] MSE: 504.355, R²: -0.165 x_train_count: 408
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad          2         4          2
True_mid         10        43         25
True_good         1         9          7

Classification Report:
              precision    recall  f1-score   support

         bad       0.15      0.25      0.19         8
         mid       0.77      0.55      0.64        78
        good       0.21      0.41      0.27        17

    accuracy                           0.50       103
   macro avg       0.38      0.40      0.37       103
weighted avg       0.63      0.50      0.55       103

[id=1] MSE: 1839.728, R²: -0.099 x_train_count: 932
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad          4        17         17
True_mid          8        25         73
True_good         6        19         64

Classification Report:
              precision    recall  f1-score   support

         bad       0.22      0.11      0.14      

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

[id=18] MSE: 4006.266, R²: 0.202 x_train_count: 273
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad         50         3          0
True_mid          9         0          0
True_good         7         0          0

Classification Report:
              precision    recall  f1-score   support

         bad       0.76      0.94      0.84        53
         mid       0.00      0.00      0.00         9
        good       0.00      0.00      0.00         7

    accuracy                           0.72        69
   macro avg       0.25      0.31      0.28        69
weighted avg       0.58      0.72      0.65        69

[id=19] MSE: 958.199, R²: 0.427 x_train_count: 873
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad         61         9          2
True_mid         20       112         14
True_good         0         1          0

Classification Report:
              precision    recall  f1-score   support

         bad       0.75      0.85      0.80      

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\User

              precision    recall  f1-score   support

         bad       0.21      0.21      0.21        28
         mid       0.51      0.53      0.52        47
        good       0.27      0.21      0.24        14

    accuracy                           0.38        89
   macro avg       0.33      0.32      0.32        89
weighted avg       0.38      0.38      0.38        89

[id=35] MSE: 1934.569, R²: 0.000 x_train_count: 30
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad          0         0          0
True_mid          4         1          3
True_good         0         0          0

Classification Report:
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         0
         mid       1.00      0.12      0.22         8
        good       0.00      0.00      0.00         0

    accuracy                           0.12         8
   macro avg       0.33      0.04      0.07         8
weighted avg       1.00      0.12   

...

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

              precision    recall  f1-score   support

         bad       0.67      0.82      0.73        22
         mid       0.50      0.23      0.32        13
        good       0.25      0.50      0.33         2

    accuracy                           0.59        37
   macro avg       0.47      0.52      0.46        37
weighted avg       0.59      0.59      0.57        37

[id=209] MSE: 502.103, R²: -0.945 x_train_count: 214
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad          0         4          4
True_mid          2        22         11
True_good         0         8          3

Classification Report:
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         8
         mid       0.65      0.63      0.64        35
        good       0.17      0.27      0.21        11

    accuracy                           0.46        54
   macro avg       0.27      0.30      0.28        54
weighted avg       0.45      0.46 

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

[id=216] MSE: 0.000, R²: 1.000 x_train_count: 246
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad          0         0          0
True_mid          0        62          0
True_good         0         0          0

Classification Report:
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00         0
         mid       1.00      1.00      1.00        62
        good       0.00      0.00      0.00         0

    accuracy                           1.00        62
   macro avg       0.33      0.33      0.33        62
weighted avg       1.00      1.00      1.00        62

[id=217] MSE: 8039.451, R²: -2.616 x_train_count: 198
Confusion Matrix:
           Pred_bad  Pred_mid  Pred_good
True_bad         10         5          4
True_mid         10         3          5
True_good         9         4          0

Classification Report:
              precision    recall  f1-score   support

         bad       0.34      0.53      0.42     

C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\82102\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is